In [2]:
import pandas as pd
import numpy as np

Personal loans are a lucrative revenue stream for banks. The typical interest rate of a two-year loan in the United Kingdom is around 10%. This might not sound like a lot, but in September 2022 alone UK consumers borrowed around £1.5 billion, which would mean approximately £300 million in interest generated by banks over two years!

![Piggy bank](piggy_bank.jpg)

Personal loans are a lucrative revenue stream for banks. The typical interest rate of a two-year loan in the United Kingdom is [around 10%](https://www.experian.com/blogs/ask-experian/whats-a-good-interest-rate-for-a-personal-loan/). This might not sound like a lot, but in September 2022 alone UK consumers borrowed [around £1.5 billion](https://www.ukfinance.org.uk/system/files/2022-12/Household%20Finance%20Review%202022%20Q3-%20Final.pdf), which would mean approximately £300 million in interest generated by banks over two years!

You have been asked to work with a bank to clean the data they collected as part of a recent marketing campaign, which aimed to get customers to take out a personal loan. They plan to conduct more marketing campaigns going forward so would like you to ensure it conforms to the specific structure and data types that they specify so that they can then use the cleaned data you provide to set up a PostgreSQL database, which will store this campaign's data and allow data from future campaigns to be easily imported. 

They have supplied you with a csv file called `"bank_marketing.csv"`, which you will need to clean, reformat, and split the data, saving three final csv files. Specifically, the three files should have the names and contents as outlined below:

## `client.csv`

| column | data type | description | cleaning requirements |
|--------|-----------|-------------|-----------------------|
| `client_id` | `integer` | Client ID | N/A |
| `age` | `integer` | Client's age in years | N/A |
| `job` | `object` | Client's type of job | Change `"."` to `"_"` |
| `marital` | `object` | Client's marital status | N/A |
| `education` | `object` | Client's level of education | Change `"."` to `"_"` and `"unknown"` to `np.NaN` |
| `credit_default` | `bool` | Whether the client's credit is in default | Convert to `boolean` data type:<br> `1` if `"yes"`, otherwise `0` |
| `mortgage` | `bool` | Whether the client has an existing mortgage (housing loan) | Convert to boolean data type:<br> `1` if `"yes"`, otherwise `0` |

<br>

## `campaign.csv`

| column | data type | description | cleaning requirements |
|--------|-----------|-------------|-----------------------|
| `client_id` | `integer` | Client ID | N/A |
| `number_contacts` | `integer` | Number of contact attempts to the client in the current campaign | N/A |
| `contact_duration` | `integer` | Last contact duration in seconds | N/A |
| `previous_campaign_contacts` | `integer` | Number of contact attempts to the client in the previous campaign | N/A |
| `previous_outcome` | `bool` | Outcome of the previous campaign | Convert to boolean data type:<br> `1` if `"success"`, otherwise `0`. |
| `campaign_outcome` | `bool` | Outcome of the current campaign | Convert to boolean data type:<br> `1` if `"yes"`, otherwise `0`. |
| `last_contact_date` | `datetime` | Last date the client was contacted | Create from a combination of `day`, `month`, and a newly created `year` column (which should have a value of `2022`); <br> **Format =** `"YYYY-MM-DD"` |

<br>

## `economics.csv`

| column | data type | description | cleaning requirements |
|--------|-----------|-------------|-----------------------|
| `client_id` | `integer` | Client ID | N/A |
| `cons_price_idx` | `float` | Consumer price index (monthly indicator) | N/A |
| `euribor_three_months` | `float` | Euro Interbank Offered Rate (euribor) three-month rate (daily indicator) | N/A |

In [33]:
df = pd.read_csv("bank_marketing.csv")
df.head()

,client_id,age,job,marital,education,credit_default,mortgage,month,day,contact_duration,number_contacts,previous_campaign_contacts,previous_outcome,cons_price_idx,euribor_three_months,campaign_outcome
0,0,56,housemaid,married,basic.4y,no,no,may,13,261,1,0,nonexistent,93.994,4.857,no
1,1,57,services,married,high.school,unknown,no,may,19,149,1,0,nonexistent,93.994,4.857,no
2,2,37,services,married,high.school,no,yes,may,23,226,1,0,nonexistent,93.994,4.857,no
3,3,40,admin.,married,basic.6y,no,no,may,27,151,1,0,nonexistent,93.994,4.857,no
4,4,56,services,married,high.school,no,no,may,3,307,1,0,nonexistent,93.994,4.857,no


In [34]:
df["job"]=df["job"].str.replace(".","_",regex=False)
df["job"]

0          housemaid
1           services
2           services
3             admin_
4           services
            ...     
41183        retired
41184    blue-collar
41185        retired
41186     technician
41187        retired
Name: job, Length: 41188, dtype: object

In [35]:
# Replace '.' with '_'
df["education"]=df["education"].str.replace(".","_",regex=False)
# Replace 'unknown' with np.NaN
df["education"]=df["education"].replace("unknown",np.NaN)

In [36]:
# Convert to boolean: 1 if 'yes', otherwise 0
df["credit_default"] = df["credit_default"].apply(lambda x: 1 if x == "yes" else 0)

# Convert to boolean data type
df["credit_default"] = df["credit_default"].astype(bool)

In [37]:
# Convert to boolean: 1 if 'yes', otherwise 0
df['mortgage']=df['mortgage'].apply(lambda x: 1 if x=="yes" else  0)
# Convert to boolean data type
df['mortgage']=df['mortgage'].astype(bool)

In [38]:

client=df[["client_id","age","job","marital","education","credit_default","mortgage"]]

In [39]:
client.head()

,client_id,age,job,marital,education,credit_default,mortgage
0,0,56,housemaid,married,basic_4y,False,False
1,1,57,services,married,high_school,False,False
2,2,37,services,married,high_school,False,True
3,3,40,admin_,married,basic_6y,False,False
4,4,56,services,married,high_school,False,False


In [40]:
client.dtypes

client_id          int64
age                int64
job               object
marital           object
education         object
credit_default      bool
mortgage            bool
dtype: object

In [11]:
client.to_csv("client.csv")

In [12]:
## campaign.csv

In [41]:
# Convert to boolean: 1 if 'success', otherwise 0
df["previous_outcome"] = df["previous_outcome"].apply(lambda x: 1 if x == "success" else 0)

# Convert to boolean data type
df["previous_outcome"] = df["previous_outcome"].astype(bool)

In [42]:
# Convert to boolean: 1 if 'yes', otherwise 0
df["campaign_outcome"] = df["campaign_outcome"].apply(lambda x: 1 if x == "yes" else 0)

# Convert to boolean data type
df["campaign_outcome"] = df["campaign_outcome"].astype(bool)

In [45]:
df["campaign_outcome"].unique()

array([False,  True])

In [44]:
df['month'].unique()

array(['may', 'jun', 'jul', 'aug', 'oct', 'nov', 'dec', 'mar', 'apr',
       'sep'], dtype=object)

In [46]:
# Month mapping dictionary
month_map = {
    'jan': 1, 'feb': 2, 'mar': 3, 'apr': 4,
    'may': 5, 'jun': 6, 'jul': 7, 'aug': 8,
    'sep': 9, 'oct': 10, 'nov': 11, 'dec': 12
}
# Convert 'month' column from string to datetime object and extract month number
df['month']=df['month'].map(month_map)

In [47]:
## start
df["year"]=2024


# Create the 'last_contact_date' column from 'day', 'month', and 'year'
df['last_contact_date'] = pd.to_datetime(df[['year', 'month', 'day']])

# Format 'last_contact_date' as "YYYY-MM-DD"
df['last_contact_date'] = df['last_contact_date'].dt.strftime('%Y-%m-%d')

In [48]:
campaign=df[["client_id","contact_duration","previous_campaign_contacts","previous_outcome","campaign_outcome","last_contact_date"]]



In [49]:
campaign.head()

,client_id,contact_duration,previous_campaign_contacts,previous_outcome,campaign_outcome,last_contact_date
0,0,261,0,False,False,2024-05-13
1,1,149,0,False,False,2024-05-19
2,2,226,0,False,False,2024-05-23
3,3,151,0,False,False,2024-05-27
4,4,307,0,False,False,2024-05-03


In [26]:
campaign.to_csv('campaign.csv')

In [30]:
campaign.dtypes

client_id                      int64
contact_duration               int64
previous_campaign_contacts     int64
previous_outcome              object
campaign_outcome              object
last_contact_date             object
dtype: object

In [27]:
economics=df[["client_id","cons_price_idx","euribor_three_months"]]

In [28]:
economics.head()

,client_id,cons_price_idx,euribor_three_months
0,0,93.994,4.857
1,1,93.994,4.857
2,2,93.994,4.857
3,3,93.994,4.857
4,4,93.994,4.857


In [29]:
economics.to_csv('economics.csv')